In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Veri Yükleme
train_data_dir = "./tumor/training"
test_data_dir = "./tumor/testing"
img_width, img_height = 224, 224

# Veri Artırma (Data Augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

# VGG16 modelini kullanarak yeni modeli oluşturun
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Mevcut modeli (Sequential) oluşturun
model = Sequential()
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Add Dropout layer to reduce overfitting
model.add(Dense(4, activation='softmax'))

# Modeli derleyin
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model Eğitimi
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32
)

# Eğitim süreci sırasında kayıp ve doğruluk değerlerini alın
loss = history.history['loss']
accuracy = history.history['accuracy']

# Doğrulama (validation) seti üzerindeki kayıp ve doğruluk değerlerini alın (varsa)
val_loss = history.history.get('val_loss')
val_accuracy = history.history.get('val_accuracy')

# Eğitim kaybını (loss) grafiğe çizin
plt.plot(loss, label='Training Loss')
# Doğrulama kaybını (loss) grafiğe çizin (varsa)
if val_loss:
    plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Doğruluk (accuracy) grafiğini çizin
plt.plot(accuracy, label='Training Accuracy')
# Doğrulama doğruluğunu (accuracy) grafiğe çizin (varsa)
if val_accuracy:
    plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Eğitilen modeli kaydetmek istiyorsanız:
model.save("tumor_vgg16_model.h5")


Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
Epoch 1/50
178/178 [==============================] - 4899s 28s/step - loss: 0.6165 - accuracy: 0.7574 - val_loss: 0.4565 - val_accuracy: 0.8820
Epoch 2/50
178/178 [==============================] - 5137s 29s/step - loss: 0.2569 - accuracy: 0.9113 - val_loss: 0.1828 - val_accuracy: 0.9352
Epoch 3/50
178/178 [==============================] - 75588s 427s/step - loss: 0.1794 - accuracy: 0.9363 - val_loss: 0.1114 - val_accuracy: 0.9586
Epoch 4/50
178/178 [==============================] - 5039s 28s/step - loss: 0.1144 - accuracy: 0.9611 - val_loss: 0.1755 - val_accuracy: 0.9445
Epoch 5/50
178/178 [==============================] - 5235s 29s/step - loss: 0.1174 - accuracy: 0.9618 - val_loss: 0.1040 - val_accuracy: 0.9648
Epoch 6/50
178/178 [==============================] - 31363s 177s/step - loss: 0.0897 - accuracy: 0.9715 - val_loss: 0.0606 - val_accuracy: 0.9828
Epoch 7/50
178/178 [======================